In [1]:
# Code based on https://github.com/alexander-held/PyHEP-2021-cabinetry/blob/main/talk.ipynb
import cabinetry
cabinetry.set_logging()

In [2]:
config = {
   "General":{
      "Measurement": "CabinetryHZZAnalysis",
      "POI": "Signal_norm",
      "InputPath": "4lep/{SamplePaths}",
      "HistogramFolder": "histograms/"
   }
}

In [3]:
bin_list = list(range(80, 255, 5))

In [4]:
config.update({
   "Regions":[
      {
         "Name": "Signal_region",
         "Filter": "(lep_charge[:,0] + lep_charge[:,1] + lep_charge[:,2] + lep_charge[:,3] == 0) & ((lep_type[:,0] + lep_type[:,1] + lep_type[:,2] + lep_type[:,3] == 44) | (lep_type[:,0] + lep_type[:,1] + lep_type[:,2] + lep_type[:,3] == 48) | (lep_type[:,0] + lep_type[:,1] + lep_type[:,2] + lep_type[:,3] == 52))",
         "Variable": "sqrt((lep_E[:,0] + lep_E[:,1] + lep_E[:,2] + lep_E[:,3])**2 - (lep_pt[:,0]*cos(lep_phi[:,0]) + lep_pt[:,1]*cos(lep_phi[:,1]) + lep_pt[:,2]*cos(lep_phi[:,2]) + lep_pt[:,3]*cos(lep_phi[:,3]))**2 - (lep_pt[:,0]*sin(lep_phi[:,0]) + lep_pt[:,1]*sin(lep_phi[:,1]) + lep_pt[:,2]*sin(lep_phi[:,2]) + lep_pt[:,3]*sin(lep_phi[:,3]))**2 - (lep_pt[:,0]*sinh(lep_eta[:,0]) + lep_pt[:,1]*sinh(lep_eta[:,1]) + lep_pt[:,2]*sinh(lep_eta[:,2]) + lep_pt[:,3]*sinh(lep_eta[:,3]))**2)/1000",
         "Binning": bin_list
      }
   ]
})

In [5]:
config.update({
   "Samples":[
      {
         "Name": "Data",
         "Tree": "mini",
         "SamplePaths": ["Data/data_A.4lep.root",
                         "Data/data_B.4lep.root",
                         "Data/data_C.4lep.root",
                         "Data/data_D.4lep.root"],
         "Data": True
      },
      {
         "Name": "Signal",
         "Tree": "mini",
         "SamplePaths": ['MC/mc_345060.ggH125_ZZ4lep.4lep.root',
                         'MC/mc_344235.VBFH125_ZZ4lep.4lep.root',
                         'MC/mc_341964.WH125_ZZ4lep.4lep.root',
                         'MC/mc_341947.ZH125_ZZ4lep.4lep.root'],
         "Weight": "((channelNumber == 345060)*2.1605e-6 + (channelNumber == 344235)*1.2588e-6 + (channelNumber == 341964)*2.5228e-5 + (channelNumber == 341947)*1.4283e-7)*mcWeight*scaleFactor_PILEUP*scaleFactor_ELE*scaleFactor_MUON*scaleFactor_LepTRIGGER"
      },
      {
         "Name": "Background $ZZ^{star}$",
         "Tree": "mini",
         "SamplePaths": "MC/mc_363490.llll.4lep.root",
         "Weight": "0.0016685*mcWeight*scaleFactor_PILEUP*scaleFactor_ELE*scaleFactor_MUON*scaleFactor_LepTRIGGER"
      },
      {
         "Name": "Background $Z,tt^{bar}$",
         "Tree": "mini",
         "SamplePaths": ['MC/mc_361106.Zee.4lep.root',
                         'MC/mc_361107.Zmumu.4lep.root',
                         'MC/mc_410000.ttbar_lep.4lep.root'],
         "Weight": "((channelNumber == 361106)*1.2980e-4 + (channelNumber == 361107)*1.3239e-4 + (channelNumber == 410000)*0.091663)*mcWeight*scaleFactor_PILEUP*scaleFactor_ELE*scaleFactor_MUON*scaleFactor_LepTRIGGER"
      }
   ]
})

In [6]:
config.update({"Systematics": []})

In [7]:
config.update({
   "NormFactors":[
      {
         "Name": "Signal_norm",
         "Samples": "Signal",
         "Nominal": 1,
         "Bounds": [0, 10]
      },
      {
         "Name": "ZZ_norm",
         "Samples": "Background $ZZ^{star}$",
         "Nominal": 1,
         "Bounds": [0, 10]
      },
      {
         "Name": "Ztt_norm",
         "Samples": "Background $Z,tt^{bar}$",
         "Nominal": 1,
         "Bounds": [0, 10]
      }
   ]
})

In [8]:
cabinetry.configuration.validate(config)

True

In [9]:
cabinetry.configuration.print_overview(config)

INFO - cabinetry.configuration - the config contains:
INFO - cabinetry.configuration -   4 Sample(s)
INFO - cabinetry.configuration -   1 Regions(s)
INFO - cabinetry.configuration -   3 NormFactor(s)
INFO - cabinetry.configuration -   0 Systematic(s)


In [ ]:
#cabinetry.template_builder.create_histograms(config, method="uproot")
cabinetry.template_builder.create_histograms(config, method="coffea")

DEBUG - cabinetry.route -   in region Signal_region
DEBUG - cabinetry.route -     reading sample Data
DEBUG - cabinetry.route -       variation Nominal


Preprocessing:   0%|          | 0/4 [00:00<?, ?file/s]

Processing:   0%|          | 0/4 [00:00<?, ?chunk/s]

In [10]:
plot_config = {
   "Regions":[
      {
         "Name": "Signal_region",
         "Variable": "4-lepton invariant mass $\mathrm{m_{4l}}$ [GeV]",
         "Binning": bin_list
      }
   ]
}

In [11]:
ws = cabinetry.workspace.build(config)
model, data = cabinetry.model_utils.model_and_data(ws)
cabinetry.visualize.data_MC(model, data, config=plot_config)

INFO - cabinetry.workspace - building workspace
WARNING - cabinetry.histo - the modified histogram histograms/Signal_region_Signal_Nominal_modified.npz does not exist
WARNING - cabinetry.histo - loading the un-modified histogram instead!


ERROR! Session/line number was not unique in database. History logging moved to new session 105


FileNotFoundError: [Errno 2] No such file or directory: 'histograms/Signal_region_Signal_Nominal.npz'

In [ ]:
fit_results = cabinetry.fit.fit(model, data)

In [ ]:
cabinetry.visualize.data_MC(model, data, config=plot_config, fit_results=fit_results, include_table=False)